In [ ]:
import pandas as pd

import numpy as np

import warnings



warnings.filterwarnings('ignore')

**1. K-Means Clustering**

In [ ]:
kmeans_df = pd.read_csv('/kaggle/input/final-merged-df/final_merged_df.csv')



print(kmeans_df.shape)

kmeans_df.isna().sum()

**Objective 1**



Identify regions with sparse grid infrastructure and high population density where wind farms and microgrids could be feasible. This will be determined by analyzing population density projections, income distribution, and wind speed.



The **K-Means clustering algorithm** will be used for this objective.

In [ ]:
from sklearn.preprocessing import LabelEncoder



# Label encode categorical variables



# Income_Distribution

income_encoder = LabelEncoder()

kmeans_df['Income_Distribution'] = income_encoder.fit_transform(kmeans_df['Income_Distribution'])



# Grid_Value

grid_encoder = LabelEncoder()

kmeans_df['Grid_Value'] = grid_encoder.fit_transform(kmeans_df['Grid_Value'])

In [ ]:
from sklearn.preprocessing import StandardScaler



# Normalize clustering features



# Clustering features

features = kmeans_df[['Pop_Density_2020', 'Wind_Speed', 'Latitude', 'Longitude', 'Income_Distribution', 'Grid_Value']]



scaler = StandardScaler()

scaled_features = scaler.fit_transform(features)


In [ ]:
from sklearn.cluster import KMeans

import matplotlib.pyplot as plt



# Finding optimal number of clusters

inertia = []

for k in range(1, 10):

    kmeans = KMeans(n_clusters=k, random_state=0)

    kmeans.fit(scaled_features)

    inertia.append(kmeans.inertia_)



# Plot elbow graph

plt.plot(range(1, 10), inertia, marker='o')

plt.xlabel('Number of clusters')

plt.ylabel('Inertia')

plt.show()

From the above visualization, using 2 clusters is the optimal choice for segmenting regions based on grid infrastructure sparsity and wind microgrid feasibility.

In [ ]:
# Fit with optimal number of clusters

kmeans = KMeans(n_clusters=2, random_state=42)

kmeans_df['Cluster'] = kmeans.fit_predict(scaled_features)



kmeans_df[['Latitude', 'Longitude', 'Pop_Density_2020', 'Wind_Speed', 'Grid_Value', 'Income_Distribution', 'Cluster']]

kmeans_df.head()

**Explore cluster characteristics**

In [ ]:
# Group by the cluster and calculate the mean of each feature

cluster_summary = kmeans_df.groupby('Cluster')[['Pop_Density_2020', 'Wind_Speed', 'Grid_Value', 'Income_Distribution']].mean()



# Print cluster summary

print(cluster_summary)


**Findings**



1. *Cluster 0*



* The areas in cluster 0 are **relatively densely populated** with approximately 95.19 people per square kilometers.



* The wind conditions in this cluster are slightly stronger than in Cluster 1, with an average speed of 6.14 m/s at 100 meters above ground level. Therefore, these regions are suitable for both **wind farms and wind microgrids**.



* The grid infrastructure in Cluster 0 is significantly limited, with a value of 0.000096. Given the region's population density, there is clearly **high demand for electricity but limited distribution capacity**.



* The average income distribution of Cluster 0 is 41.999, indicating that approximately **50% of the population lives below the $2.15 poverty line**. Therefore, this cluster has a higher proportion of people living in poverty.





2. *Cluster 1*



* Compared to Cluster 0, the regions in Cluster 1 have a **slightly lower population density** of 94.17 people per square kilometer.



* Wind conditions in Cluster 1 are **generally calmer**, with an average wind speed of 5.80 m/s at 100 meters above ground level.



* Cluster 1 has a **higher presence of grid structures**, with a grid value of 0.5524, indicating that these regions are likely more developed compared to those in Cluster 0 and have higher access to electricity/energy.



* The average income distribution in Cluster 1 is 21.06, indicating that **only 21.06% of the population in these regions lives below the $2.15 poverty line**.

In [ ]:
from sklearn.metrics import davies_bouldin_score



kmeans_df['Cluster'] = kmeans.labels_

db_score = davies_bouldin_score(scaled_features, kmeans.labels_)

print(f"Davies-Bouldin Index: {db_score}")


A **Davies-Bouldin Index of 0.73315** indicates that the clusters are relatively compact and distinct.

In [ ]:
from sklearn.metrics import calinski_harabasz_score



kmeans_df['Cluster'] = kmeans.labels_

calinski_score = calinski_harabasz_score(scaled_features, kmeans.labels_)

print(f"Calinski-Harabasz Score: {calinski_score}")

A **Calinski-Harabasz Score of 995263.1557610929** data points in each cluster are tightly grouped together, and the clusters themselves are well-separated. 



Conclusively, this model has successfully created meaningful clusters with a significant difference between them.

In [ ]:
import pickle

# Assuming your model is named `kmeans_model`
with open("kmeans_model.pkl", "wb") as f:
    pickle.dump(kmeans, f)


In [ ]:
from flask import Flask, request, jsonify
import pickle
import numpy as np

# Load pre-trained KMeans model
with open("kmeans_model.pkl", "rb") as f:
    kmeans_model = pickle.load(f)

app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    data = request.json["data"]
    data = np.array(data).reshape(1, -1)
    cluster = kmeans_model.predict(data)
    return jsonify({"cluster": int(cluster[0])})

if __name__ == '__main__':
    app.run(host="0.0.0.0", port=5000)


**2. Random Forest**

**Objective:** Classify areas based on their suitability for microgrid installation and map regions with high wind speeds to optimize the placement of wind farms and mini-grids in off-grid locations.



Classification targets will be defined and labeled that will classify arreas based on their suitability for wind farm/microgrid installation. These labels will reflect levels of microgrid viability.



These labels are based on **renewable energy potential** ['Wind_Speed'] thresholds. The target labels are as follows:



1. **Viable:** *Areas with wind speed of more than or equal to 5 m/s 100m above ground*.



2. **Not Viable:** *Areas with wind speed of less than or equal to 5 m/s 100m above ground*.


In [ ]:
rforest_df = pd.read_csv(r'C:\Users\pc\Documents\Projects\Energy_Access_and_Electrification_Planning_in_Kenya\final_merged_df.csv')

rforest_df.head()

In [ ]:
# Define microgrid suitability as binary based on wind speed thresholds

rforest_df['Microgrid_Suitability'] = rforest_df['Wind_Speed'].apply(lambda x: 1 if x >= 5 else 0)



# Check the result

rforest_df[['Wind_Speed', 'Microgrid_Suitability']].head()


In [ ]:
rforest_df.Microgrid_Suitability.value_counts()

The y variable is highly imbalanced.

In [ ]:
# Categorical variables

from sklearn.preprocessing import LabelEncoder



# Income_Distribution: OHE to avoid imposing any ordinal relationships

rforest_df = pd.get_dummies(rforest_df, columns=['Income_Distribution'], drop_first=False, dtype=int)



# Grid_Value: Label Encoding since the grid values are discrete categories

label_encoder = LabelEncoder()

rforest_df['Grid_Value_Encoded'] = label_encoder.fit_transform(rforest_df['Grid_Value'])



rforest_df.head()

In [ ]:
# Label X and y



y = rforest_df['Microgrid_Suitability']

X = rforest_df.drop(columns= ['Microgrid_Suitability', 'Grid_Value'])

In [ ]:
from sklearn.model_selection import train_test_split



# Train-test split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler



# Feature scaling

scaler = StandardScaler()



# Fit only on training data, then transform both training and test data

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier



# Initialize and train the Random Forest model

model = RandomForestClassifier(class_weight="balanced", random_state=42)

model.fit(X_train_scaled, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix



# Predict on the test set

y_pred = model.predict(X_test_scaled)



# Calculate accuracy

accuracy = accuracy_score(y_test, y_pred)

print(f"Model Accuracy: {accuracy:.2f}")



# Classification report for more detailed metrics

print("Classification Report:")

print(classification_report(y_test, y_pred))



# Confusion matrix

print("Confusion Matrix:")

print(confusion_matrix(y_test, y_pred))

In [ ]:
from sklearn.model_selection import cross_val_score



# Initialize the Random Forest model with chosen parameters

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)



# Perform 5-fold cross-validation

cv_scores = cross_val_score(rf_model, X, y, cv=5, scoring='accuracy')



print("Cross-Validation Scores:", cv_scores)

print("Mean CV Accuracy:", np.mean(cv_scores))

print("Standard Deviation of CV Accuracy:", np.std(cv_scores))

**3. hdbscan**

In [ ]:
import pandas as pd

hdbscan_df = pd.read_csv(r'C:\Users\pc\Documents\Projects\Energy_Access_and_Electrification_Planning_in_Kenya\final_merged_df.csv')

hdbscan_df.head()

In [ ]:
# One-hot encode 'Income_Distribution'

hdbscan_df = pd.get_dummies(hdbscan_df, columns=['Income_Distribution'], prefix='Income')



income_columns = [col for col in hdbscan_df.columns if col.startswith('Income_')]

clustering_data = hdbscan_df[['Pop_Density_2020', 'Wind_Speed', 'Latitude', 'Longitude', 'Grid_Value'] + income_columns]

In [ ]:
from sklearn.preprocessing import StandardScaler



# Standardize the data to bring all features to a similar scale

scaler = StandardScaler()

clustering_data_scaled = scaler.fit_transform(clustering_data)

In [ ]:
from sklearn.decomposition import PCA



# Apply PCA to reduce to a manageable number of components

pca = PCA(n_components=2)  # Adjust to 2 components for efficient clustering

clustering_data_reduced = pca.fit_transform(clustering_data_scaled)

In [ ]:
import hdbscan



# Apply HDBSCAN with optimized parameters

hdbscan_clusterer = hdbscan.HDBSCAN(min_samples=10, min_cluster_size=50)  # Adjust min_samples and min_cluster_size as needed

clusters = hdbscan_clusterer.fit_predict(clustering_data_reduced)

In [ ]:
# Filter out noise points (-1 label in HDBSCAN) before calculating metrics

clustered_data = clustering_data_reduced[clusters != -1]

valid_clusters = clusters[clusters != -1]

In [ ]:
# Access cluster stability scores from HDBSCAN

stability_scores = hdbscan_clusterer.probabilities_

In [ ]:
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score



if len(set(valid_clusters)) > 1:  # Ensure there's more than one cluster for evaluation

    db_index = davies_bouldin_score(clustered_data, valid_clusters)

    ch_index = calinski_harabasz_score(clustered_data, valid_clusters)

    print("Davies-Bouldin Index:", db_index)

    print("Calinski-Harabasz Index:", ch_index)

else:

    print("Insufficient clusters for evaluation metrics")



# Print stability scores and cluster labels

print("Cluster Labels:", clusters)

print("Cluster Stability Scores:", stability_scores)

In [ ]:
import matplotlib.pyplot as plt



# Visualize clustering results

plt.figure(figsize=(10, 6))

plt.scatter(clustering_data_reduced[:, 0], clustering_data_reduced[:, 1], c=clusters, cmap='viridis', s=5)

plt.colorbar(label='Cluster')

plt.xlabel('PCA Component 1')

plt.ylabel('PCA Component 2')

plt.title('HDBSCAN Clustering Results on Full Dataset')

plt.show()

**4. XGBoost**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier


In [ ]:
xgboost_df = kmeans_df.copy()
print(xgboost_df.shape)
print(xgboost_df.info())
xgboost_df.head()

In [ ]:

X = xgboost_df.drop('Cluster', axis=1)  
y = xgboost_df['Cluster']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:

xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)



xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:")
print(classification_report(y_test, y_pred))

**Cross-Validation**

In [ ]:
from sklearn.model_selection import cross_val_score, GridSearchCV
from xgboost import XGBClassifier

xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
cv_score = cross_val_score(xgb_model, X_train, y_train, cv=5, scoring='accuracy')
print(f"Cross-validation Accuracy: {cv_score.mean():.4f}")

**HyperParameter Tuning**

In [ ]:
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [50, 100, 200],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9]
}

grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Cross-validation Score: {grid_search.best_score_:.4f}")

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Predict the labels for the test set
y_pred = model.predict(X_test)

# Convert probabilities to the class with the highest probability
y_pred_classes = np.argmax(y_pred, axis=1)

# Convert y_test from one-hot to single-label format
y_test_classes = np.argmax(y_test, axis=1)

# Compute the confusion matrix
cm = confusion_matrix(y_test_classes, y_pred_classes)

# Plot the confusion matrix
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
neuralnetwork_df = kmeans_df.copy()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

neuralnetwork_df = kmeans_df.copy()

X = neuralnetwork_df.drop(columns=['Cluster'])  # Features
y = neuralnetwork_df['Cluster']  # Target (Cluster)

# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# One-hot encode the target variable if it is categorical
y_one_hot = to_categorical(y)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_one_hot, test_size=0.2, random_state=42)

# Define the deep learning model
model = Sequential()

# Input Layer (26 features in input)
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))

# Hidden Layers
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))

# Output Layer (Number of classes in the target)
model.add(Dense(y_one_hot.shape[1], activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=256, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy * 100:.2f}%')

# You can use the model to predict on new data as well
predictions = model.predict(X_test)


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Predict the labels for the test set
y_pred = model.predict(X_test)

# Convert probabilities to the class with the highest probability
y_pred_classes = np.argmax(y_pred, axis=1)

# Convert y_test from one-hot to single-label format
y_test_classes = np.argmax(y_test, axis=1)

# Compute the confusion matrix
cm = confusion_matrix(y_test_classes, y_pred_classes)

# Plot the confusion matrix
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
from sklearn.metrics import classification_report
import numpy as np


y_test_classes = np.argmax(y_test, axis=1)


print(classification_report(y_test_classes, y_pred_classes))


In [ ]:
from sklearn.metrics import roc_curve, auc
import numpy as np
import matplotlib.pyplot as plt

# Convert y_test and y_pred from one-hot to single-label format (for binary classification)
y_test_classes = np.argmax(y_test, axis=1)
y_pred_prob = y_pred[:, 1]  # Assuming class 1 is the positive class, adjust if needed

# Calculate the ROC curve
fpr, tpr, thresholds = roc_curve(y_test_classes, y_pred_prob)
roc_auc = auc(fpr, tpr)

# Plot the ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()


In [ ]:
neuralnetwork_df.info()

In [ ]:
# Convert Grid_Value to categorical (ordinal encoding)
neuralnetwork_df['Grid_Value'] = neuralnetwork_df['Grid_Value'].astype('category')

# One-hot encoding for categorical Grid_Value
neuralnetwork_df = pd.get_dummies(neuralnetwork_df, columns=['Grid_Value'], drop_first=True, dtype='int')  
neuralnetwork_df.head()

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

X = neuralnetwork_df.copy()  
y = neuralnetwork_df['Grid_Value']                 

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


**transformation model**

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
X = neuralnetwork_df.copy() 
# Scale the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Define input shape based on the number of features
input_dim = X.shape[1]

# Autoencoder model with additional dropout and batch normalization
input_layer = Input(shape=(input_dim,))
encoded = Dense(128, activation='relu')(input_layer)
encoded = BatchNormalization()(encoded)
encoded = Dropout(0.2)(encoded)
encoded = Dense(64, activation='relu')(encoded)
encoded = BatchNormalization()(encoded)
encoded = Dropout(0.2)(encoded)
encoded = Dense(16, activation='relu')(encoded)  # Compressed representation

decoded = Dense(64, activation='relu')(encoded)
decoded = BatchNormalization()(decoded)
decoded = Dropout(0.2)(decoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = BatchNormalization()(decoded)
decoded = Dropout(0.2)(decoded)
decoded = Dense(input_dim, activation='linear')(decoded)

# Define the autoencoder model
autoencoder = Model(inputs=input_layer, outputs=decoded)
encoder = Model(inputs=input_layer, outputs=encoded)  # Encoder-only model for transformed data

# Compile the model with a lower learning rate
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# Train the autoencoder with adjusted batch size
history = autoencoder.fit(X_scaled, X_scaled, epochs=50, batch_size=64, validation_split=0.2)

# Generate transformed data using the encoder
X_transformed = encoder.predict(X_scaled)


In [ ]:
neuralnetwork_df['Grid_Value'].value_counts()

In [ ]:
kmeans_df.info()

In [ ]:
neuralnetwork_df = pd.get_dummies(neuralnetwork_df, columns=['Income_Distribution'], drop_first=True, dtype='int')  

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2

# Example model architecture with regularization, dropout, and batch normalization
model = Sequential([
    Dense(128, activation='relu', input_shape=(input_shape,), kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1)  # Adjust output layer as per your target variable
])

# Compile model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Fit model with early stopping
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val),
                    batch_size=32, callbacks=[early_stopping])


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2

# Example model architecture with regularization, dropout, and batch normalization
model = Sequential([
    Dense(128, activation='relu', input_shape=(input_shape,), kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1)  # Adjust output layer as per your target variable
])

# Compile model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Fit model with early stopping
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val),
                    batch_size=32, callbacks=[early_stopping])


In [ ]:
pip install Flask scikit-learn joblib gunicorn


In [ ]:
from sklearn.cluster import KMeans
import joblib


joblib.dump(kmeans, 'kmeans_model.pkl')

In [ ]:
from flask import Flask, request, jsonify
import joblib
import numpy as np

app = Flask(__name__)

# Load the saved KMeans model
model = joblib.load("kmeans_model.pkl")

@app.route('/predict', methods=['POST'])
def predict():
    # Get data from POST request
    data = request.get_json(force=True)
    
    # Assuming input data is in the form of a 2D list
    data = np.array(data['input'])
    
    # Predict cluster
    predictions = model.predict(data)
    
    # Return predictions as JSON response
    return jsonify({'predictions': predictions.tolist()})

if __name__ == '__main__':
    app.run(debug=True)
